# Introduction to Data Science with Python and Jupyter

In this class we've studied data organization, called schema, and SQL that allows us to define, update and query our data. That's a solid start to Data Science but we need to be able to do more. SQL has a very limited ability to get statistical information from a query and no ability to do graphing. That's what we need [Jupyter](https://jupyter.org/) and [Pandas](https://pandas.pydata.org/) for. 

To setup the features of this notebook run the code in this cell that imports the `pandas` library. 

In [ ]:
import pandas as pd 
url = "sqlite:///population.sqlite3"

Pandas works by creating a `DataFrame`, which is just a table full of data. There are many ways to load a `DataFrame`, including using an SQL query. The cell below runs the query inside of the quotes and loads the result into a `DataFrame` named `df`. It then displays the table.

In [ ]:
query = """
select * from population;
"""

df = pd.read_sql_query(query, url)
display(df)

DataFrames expect to hold a *series* of data and some features will only work if you tell Pandas what the `index` (i.e. the X-axis of your data is). The cell below uses an SQL query to get California's population over the time period in our data set. Notice that there is no longer an index column with row numbers, the `year` column takes its place.  

In [ ]:
query = """
select year, sum(pop_male), sum(pop_female), sum(pop_total)
    from population
    group by year;
"""

df = pd.read_sql_query(query, url, index_col='year')
display(df)

Indexing enables making graphs and Pandas has many graph types that you can choose from. 

In [ ]:
query = """
select year, sum(pop_male), sum(pop_female), sum(pop_total)
    from population
    group by year;
"""

df = pd.read_sql_query(query, url, index_col='year')
display(df.plot(figsize=(10,6)))

Sometimes data is surpising. In class you produced a query that showed the average age of the population. That's an interesting snapshot of a population, but in order to understand change over time you need a plot. Here's a graph of the average age of the population of California.

In [ ]:
query = """
select year, sum((age + 0.5) * pop_total) / sum(pop_total) as `Average Age`
    from population
    group by year;
"""

df = pd.read_sql_query(query, url, index_col='year')
display(df.plot(figsize=(10,6)))

> **Why did the average age stay stable until the mid 90s?**

Pandas has a lot of functionality that's also in SQL and some that isn't. For example if you want to see counties represented against each other you can use the `pivot` function of a DataFrame. 

In [ ]:
query = """
select year, county, sum((age + 0.5) * pop_total) / sum(pop_total) as `Average Age`
    from population
    where 
        county = 'SANTA CRUZ'
        or county = 'LOS ANGELES'
        or county = 'MARIN'
    group by year, county;
"""

df = pd.read_sql_query(query, url, index_col='year')
display(df.pivot(columns='county').plot(figsize=(10,6)))

School funding has been a generational problem in Santa Cruz, with Cabrillo seeing a decline in enrollment for the last 20 years. This graph helps us understand that problem.

In [ ]:
query = """
select year, sum(pop_total) as `Minors`
    from population
    where 
        county = 'SANTA CRUZ'
        and age < 18
    group by year;
"""

df = pd.read_sql_query(query, url, index_col='year')
display(df.plot(figsize=(10,6)))

It seems that in 1996 there were approximately 65,000 minors in Santa Cruz county. That number declined rapidly to a low of about 54,000, **a 17% decrease** by 2010. Fewer local school kids means fewer Cabrillo students. There are other ways we can look at demographics. Here's a histogram that shows the population of Santa Cruz by age.  

In [ ]:
query = """
select age, sum(pop_total)
    from population
    where 
        county = 'SANTA CRUZ'
        and year = '2018'
    group by age;
"""

df = pd.read_sql_query(query, url, index_col='age')
display(df.plot.bar(rot=90, figsize=(20,6)))